In [18]:

import geopandas as gpd
import holoviews as hv
import numpy as np
import pandas as pd
import pyproj
import stackstac
import xarray as xr
from shapely.geometry import mapping
import rioxarray as rio


In [ ]:
# from dask.distributed import Client, progress, LocalCluster
# cluster = LocalCluster()
# client = Client(cluster)

In [2]:
aoi = gpd.read_file('data/catchment_outline.geojson', crs="EPGS:4326")

In [3]:
aoi_geojson = mapping(aoi.iloc[0].geometry)

In [4]:
import pystac_client

URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)

In [5]:
items = catalog.search(
    intersects=aoi_geojson,
    collections=["sentinel-2-l2a"],
    datetime="2019-02-01/2019-06-10"
).item_collection()
len(items)

101

In [6]:
selected_item = items[1]

In [7]:
for key, asset in selected_item.assets.items():
    print(f"{key}: {asset.title}")

aot: Aerosol optical thickness (AOT)
blue: Blue (band 2) - 10m
coastal: Coastal aerosol (band 1) - 60m
granule_metadata: None
green: Green (band 3) - 10m
nir: NIR 1 (band 8) - 10m
nir08: NIR 2 (band 8A) - 20m
nir09: NIR 3 (band 9) - 60m
red: Red (band 4) - 10m
rededge1: Red edge 1 (band 5) - 20m
rededge2: Red edge 2 (band 6) - 20m
rededge3: Red edge 3 (band 7) - 20m
scl: Scene classification map (SCL)
swir16: SWIR 1 (band 11) - 20m
swir22: SWIR 2 (band 12) - 20m
thumbnail: Thumbnail image
tileinfo_metadata: None
visual: True color image
wvp: Water vapour (WVP)
aot-jp2: Aerosol optical thickness (AOT)
blue-jp2: Blue (band 2) - 10m
coastal-jp2: Coastal aerosol (band 1) - 60m
green-jp2: Green (band 3) - 10m
nir-jp2: NIR 1 (band 8) - 10m
nir08-jp2: NIR 2 (band 8A) - 20m
nir09-jp2: NIR 3 (band 9) - 60m
red-jp2: Red (band 4) - 10m
rededge1-jp2: Red edge 1 (band 5) - 20m
rededge2-jp2: Red edge 2 (band 6) - 20m
rededge3-jp2: Red edge 3 (band 7) - 20m
scl-jp2: Scene classification map (SCL)
swi

In [8]:
ds = stackstac.stack(items)

/home/pier/miniconda3/envs/treundici_dev/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [9]:
green = ds.sel(band='green')
swir = ds.sel(band='swir16')
scl = ds.sel(band='scl')

In [10]:
ndsi = (green - swir) / (green + swir)

In [11]:
snow = xr.where((ndsi > 0.42) & ~np.isnan(ndsi), 1, ndsi)

In [12]:
snowmap = xr.where((snow <= 0.42) & ~np.isnan(snow), 0, snow)

In [13]:
# mask = (scl != 8) & (scl != 9) & (scl != 3) 
mask = np.logical_not(scl.isin([8, 9, 3]))  # more elegant but not sure about it from a teaching perspective

In [14]:
snow_cloud = xr.where(mask, snowmap, 2)

In [15]:
aoi_utm32 = aoi.to_crs(epsg=32632)

In [16]:
geom_utm32 = aoi_utm32.iloc[0]['geometry']

In [19]:
snow_cloud.rio.write_crs("EPSG:32632", inplace=True)
snow_cloud.rio.set_nodata(np.nan, inplace=True)

<xarray.DataArray 'stackstac-654bc9a921e0b18ba5e4ecf53cf76ce3' (time: 101,
                                                                y: 20982,
                                                                x: 10980)>
dask.array<where, shape=(101, 20982, 10980), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2019-02-01...
    id                                       (time) <U24 'S2B_32TPT_20190201_...
    band                                     <U12 'scl'
  * x                                        (x) float64 6e+05 ... 7.098e+05
  * y                                        (y) float64 5.3e+06 ... 5.09e+06
    updated                                  (time) <U24 '2022-11-08T12:14:57...
    ...                                       ...
    gsd                                      object None
    common_name                              object None
    center_wavelength                        object None
    full_width_half_max                      object None
    epsg                                     int64 32632
    spatial_ref                              int64 0

In [ ]:
snowmap_clipped = snow_cloud.rio.clip([geom_utm32])

In [ ]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    clipped = snowmap_clipped.compute()

In [ ]:
clipped_date = clipped.groupby(clipped.time.dt.floor('D')).max(skipna=True)

In [ ]:
clipped_date = clipped_date.rename({'floor': 'date'})

In [ ]:
clipped_date.hvplot.image(
    x='x',
    y='y',
    groupby='date',
    crs=pyproj.CRS.from_epsg(32632),
    cmap='Pastel2',
    clim=(-1, 2),
    frame_width=500,
    frame_height=500,
    title='Snowmap',
    geo=True, tiles='OSM')

In [ ]:
cloud = xr.where(clipped_date == 2, 1, np.nan).count(dim=['x', 'y'])

In [ ]:
aot_total = clipped_date.count(dim=['x', 'y'])

In [ ]:
cloud_fraction = cloud / aot_total * 100

In [ ]:
cloud_fraction.hvplot.line(title='Cloud cover %') * hv.HLine(25).opts(
    color='red',
    line_dash='dashed',
    line_width=2.0,
)

In [ ]:
snow = xr.where(clipped_date == 1, 1, np.nan).count(dim=['x', 'y'])
snow_fraction = snow / aot_total * 100

In [ ]:
snow_fraction.hvplot.line(title='Snow cover area (%)')

In [ ]:
masked_cloud_fraction = cloud_fraction < 30

In [ ]:
snow_fraction.sel(date=masked_cloud_fraction)

In [ ]:
snow_fraction.hvplot.line(title="Snow fraction")

In [ ]:
discharge_ds = pd.read_csv('data/ADO_DSC_ITH1_0025.csv', sep=',', index_col='Time', parse_dates=True)
discharge_ds.head()

In [ ]:
start_date = pd.to_datetime("2019/02/01")
end_date = pd.to_datetime("2019/06/30")
# filter discharge data to start and end dates
discharge_ds = discharge_ds.loc[start_date:end_date]

discharge_ds.discharge_m3_s.hvplot() * snow_fraction.hvplot()  

#(marker='o',ax=ax1,color='orange') #label='Discharge', xlabel='', ylabel='Discharge (m$^3$/s)',ax=ax0) 
#ax0.legend(loc='center left', bbox_to_anchor=(0, 0.6))
#ax1.set_ylabel('Snow cover area (%)')
#ax1.legend(loc='center left', bbox_to_anchor=(0, 0.5),labels=['SCA'])
#plt.show()